In [3]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [4]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = '../docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_9792/1475080136.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [6]:
print(vectordb._collection.count())

208


In [7]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [9]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [10]:
from langchain.chains import RetrievalQA

In [11]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [12]:
result = qa_chain({"query": question})

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_9792/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [13]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. In addition to these main topics, there will be discussions covering extensions of the material taught in the main lectures.'

In [ ]:
# Prompt

In [21]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [23]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [24]:
question = "Is probability a class topic?"

In [25]:
result = qa_chain({"query": question})

In [26]:
result["result"]

'Yes, probability is a topic assumed to be familiar to students in this class, as mentioned by the instructor. Thanks for asking!'

In [27]:
result["source_documents"][0]

Document(metadata={'page': 4, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page_label': '5', 'source': '../docs/MachineLearning-Lecture01.pdf', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'total_pages': 22, 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00'}, page_content="of this class will not be very programming intensive, although we will do some \nprogramming, mostly in either MATLAB or Octave. I'll say a bit more about that later.  \nI also assume familiarity with basic probability and statistics. So most undergraduate \nstatistics class, like Stat 116 taught here at Stanford, will be more than enough. I'm gonna \nassume all of you know what random variables are, that all of you know what expectation \nis, what a variance or a random variable is. And in case of some of you, it's been a while \nsince you've seen some of this material. At some of the discussion sections, we'll actually \ngo over some of the prerequisites

In [42]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [43]:
result = qa_chain_mr({"query": question})

In [44]:
result["result"]

'The prerequisites of basic probability and statistics, as well as basic linear algebra, are needed for the class because the course will involve programming tasks, mostly in MATLAB or Octave. Understanding these concepts is crucial for analyzing data, making informed decisions, and working with matrices and vectors commonly used in data analysis and machine learning algorithms. Additionally, the instructor assumes that all students have a basic knowledge of computer science and computer skills to ensure they are equipped with foundational knowledge for understanding and applying machine learning concepts effectively.'

In [41]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'The prerequisites of basic probability and statistics, as well as basic linear algebra, are needed for this class because machine learning plays a crucial role in various applications, including understanding the human genome and potentially contributing to finding a cure for diseases like cancer. By having a strong foundation in probability, statistics, and linear algebra, students will be well-equipped to apply state-of-the-art machine learning algorithms to real-world problems and potentially contribute to advancements in fields like genomics and healthcare. Understanding these mathematical concepts is essential for students to be able to effectively utilize machine learning algorithms and potentially engage in research in the field.'

In [33]:
"""
RetrievalQA limitations
QA fails to preserve conversational history.
"""

'\nRetrievalQA limitations\nQA fails to preserve conversational history.\n'

In [34]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [35]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic in the course being described. The instructor assumes familiarity with basic probability and statistics, so it will likely be covered in the context of machine learning and related algorithms.'

In [36]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites mentioned in the context are needed because the course assumes familiarity with basic concepts in probability and statistics, as well as basic linear algebra. Understanding these concepts is essential for grasping the material covered in the machine learning course.'